In [121]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import cluster
import math
import os
import re
import sklearn
from datetime import date, datetime
from davies_bouldin import davies_bouldin_score # local
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from MergeDataFrameToTable import MergeDFToTable
try:
    from sklearn.metrics import davies_bouldin_score as db2
    from sklearn.metrics.cluster import v_measure_score as v2
    print('sklearn version qualified, import db score')
except:
    print("use local davies-bouldin score instead")
    pass
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# import seaborn as sns
# os.getcwd()
import warnings
import sys 

warnings.filterwarnings('ignore')
print("Sklearn version here:", sklearn.__version__, '\t')
print("python version here:", sys.version, '\t') 
print("pandas version here:", pd.__version__, '\t') 


sklearn version qualified, import db score
Sklearn version here: 0.24.2 	
python version here: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)] 	
pandas version here: 1.4.1 	


In [4]:
def ld_dt(if_local = False):
    if if_local == True:
        link = r'C:\Users\dscshap3808\Documents\my_scripts_new\play1\ou_daily_kpi.csv'
        df = pd.read_csv(link, sep = '\001').fillna(0)
        df.columns =  [re.sub('\w+\.', '', i) for i in df.columns]
    else:
        spark = SparkSession.builder.enableHiveSupport().getOrCreate()
        spark.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')
        """
        load data
        """
        df = spark.sql("""select * 
                from dsc_dws.dws_dsc_wh_ou_daily_kpi_sum where operation_day > '20210901' """)
        # df.show(15,False)
        df = df.select("*").toPandas()
        df = df.fillna(0)
        df = df.replace([np.inf, -np.inf], 0) 
        pass
    df[[
        'inbound_receive_qty', 'outbound_shipped_qty', 'total_working_hour','operation_day'
        ]] = df[[
            'inbound_receive_qty', 'outbound_shipped_qty', 'total_working_hour','operation_day'
            ]].astype(np.int64)

    clean_df2 = df.groupby('ou_code')[[
        'inbound_receive_qty', 'outbound_shipped_qty'
        ]].sum().reset_index()
    clean_df2['sum'] = clean_df2.sum(axis = 1)
    clean_df2 = clean_df2[clean_df2['sum'] != 0]
    df = df[df['ou_code'].isin(clean_df2.ou_code)] 

    clean_df3 = (df.groupby('ou_code')[[
        'total_working_hour'
        ]].sum() == 0).reset_index()
    clean_df3 = clean_df3[clean_df3['total_working_hour'] == False]
    df = df[df['ou_code'].isin(clean_df3.ou_code)]
    df= df.reset_index()

    df = df[[
        'ou_code','operation_day', 'inbound_receive_qty', 'is_holiday',
        'outbound_shipped_qty','total_head_count','total_working_hour',
        'outsource_working_hour', 'perm_working_hour',
        'other_working_hour', 'direct_working_hour', 'indirect_working_hour',
        'outbound_inbound_qty_ratio', 'perm_working_hour_ratio',
        'working_hour_per_head', 'location_usage_rate', 'location_idle_rate']]

    df = df.fillna(0).sort_values('operation_day')
    df = df[df['total_working_hour'] != 0]
    return df

def reshape_to_easy_train(df, col = None):
    """
    reshape dataframe to array
    """
    if type(col) == str:
        return np.reshape(list(df[col]), (-1,1))
    else:
        return np.array(df[col])

In [5]:
def transformer(df,col):
    """
    unfixed stdscaler
    """
    try:
        del mx
    except:
        pass
    mx = StandardScaler()
    df[col] = mx.fit_transform(reshape_to_easy_train(df,col))
    return df
        
def mxscale_(df):
    """
    outlier remove 1
    """
    df['inef'] = df['inbound_receive_qty']  / df['total_working_hour']
    df['ouef'] = df['outbound_shipped_qty'] / df['total_working_hour']
    df['in_out_sum'] = df['outbound_shipped_qty'] + df['inbound_receive_qty']
    df['tt_wh'] = df['total_working_hour']
    for col in ['inef', 'ouef']:
        df = transformer(df,col)
    return df

def drop_outliers(df_train, col = None):
    """
    outlier remove 2
    """
    flag_high = np.mean(df_train[col]) + 1 * np.std(df_train[col])
    flag_low  = np.mean(df_train[col]) - 1 * np.std(df_train[col])
    # print(col, flag_low) 
    df_train['flag_' + col] = np.where(
        (df_train[col] >= flag_low) & (df_train[col] <= flag_high), 
        1, 
        0)
    df_train = df_train[df_train['flag_' + col] != 0]
    return df_train



In [6]:
def get_transformed_data(df_train,  mode = 'train'):
    """
    transform data
    """
    df_train = mxscale_(df_train)
    col = 'inef'
    df_train = drop_outliers(df_train= df_train, col = col) 
    del col
    col = 'ouef'
    df_train = drop_outliers(df_train = df_train, col = col) 
    # print(df_train[['inef','ouef']].tail(20))
    try:
        del mx
    except:
        pass
    mx = StandardScaler()
    mx.fit(
        reshape_to_easy_train(
            df_train,[
                'inbound_receive_qty',
                'outbound_shipped_qty',
                'in_out_sum',
                # 'tt_wh'
                ])
        )
    train_set_inout_transformed = mx.transform(
        reshape_to_easy_train(
            df_train,[
                'inbound_receive_qty',
                'outbound_shipped_qty',
                'in_out_sum',
                # 'tt_wh'
                ])
    )
    df_teste['in_out_sum'] = df_teste['inbound_receive_qty'] + df_teste['outbound_shipped_qty']
    df_teste['tt_wh'] = df_teste['total_working_hour']
    test_set_inout_transformed = mx.transform(
        reshape_to_easy_train(
            df_teste,[
                'inbound_receive_qty',
                'outbound_shipped_qty',
                'in_out_sum',
                # 'tt_wh'
                ])
        )
    if mode != 'train':
        return test_set_inout_transformed
    else:
        return train_set_inout_transformed

In [122]:

def train(df_train, col = 'combined'):
        """
        train
        """
        input = get_transformed_data(df_train, 'train')
        input2= get_transformed_data(df_train, 'teste')
        try:
                del kmeans
        except:
                pass
        # print(input.shape, input2.shape)
        scores = []
        for center in list(range(min_kernel, max_kernel)):
                # kmeans = cluster.MiniBatchKMeans(n_clusters=center, random_state=529)
                # kmeans = cluster.SpectralClustering(n_clusters=center,\
                #         affinity='nearest_neighbors',
                #         n_neighbors=4,
                #         random_state=529, 
                #         n_jobs= -1)
                kmeans = cluster.KMeans(n_clusters=center, random_state = 529)
                model  = kmeans.fit_predict(
                        X = input,
                        )
                
                if pd.Series(model).value_counts().min() <= 2:
                        print("single point in cluster! dropping, center size = %s"%center)
                        score = 100
                        # pass
                else:
                        score  = db2(
                                input,
                                model
                                )
                scores.append(score)
        print(scores,np.array(scores).mean())
        if np.array(scores).mean() == 100:
                raise ValueError('drop cc since score all 100')
        else:
                pass
        center = np.argmin(scores) + min_kernel
        print(center, 'Center count, cap is %s'%max_kernel)
        try:
                del alg1
        except:
                pass
        # alg1 = cluster.SpectralClustering(n_clusters=center,\
        #                 affinity='nearest_neighbors',
        #                 n_neighbors=4,
        #                 random_state=529, 
        #                 n_jobs= -1)
        alg1 = cluster.KMeans(n_clusters=center, random_state = 529)
        hist1 = alg1.fit(
                input 
                )

        k = hist1.predict(
                input2
                )
        return k

def median_log(x):
        return math.log(np.median(x))
def transform_inout_to_medianlog(df_teste):
        try:
                df_teste['in_out_sum_log'] = df_teste.groupby('kernels')['in_out_sum'].transform(median_log)
                df_teste['std_inout'] = (StandardScaler().fit_transform(reshape_to_easy_train(df_teste, 'in_out_sum_log'))/10 + 1)**2
        except:
                df_teste['in_out_sum_log'] = 1
                df_teste['std_inout'] = 1
        return df_teste


# \_\_main__ part of the script

In [141]:
"""
test env
"""
df0 = ld_dt(if_local = True)
min_kernel = 3
max_kernel = 11
df_final = pd.DataFrame()

cc = ['COSTASHH2S',
'COSTASHHTS',
'STARBBJZWS',
'STARBCDXXS',
'STARBBJALS',
'STARBCDCHS',
'STARBSZQAS',
'STARBSHHTS',
'STARBSYXXS',
'STARBHZSCS',
'STARBGZHPS',
'TAIHUBJALS',
'BRIDADGXSS']
# cc = ['STARBSYXXS', 'STARBSHHTS']
df = df0[df0['ou_code'].isin(cc)]
ou_code =  cc #'STARBSYXXS', 'FORDCCSLGS'

# ['COACHSHSFS', 'COSTASHHTS', 'FERRECDXXS', 'MARSXBJSHS', 'MARSXCDXXS', 'STARBSYXXS',
# 'MARSXJXXWS', 'STARBSHHTS', 'ABBOTCDXXS', 'ABBOTSHDBS', 'ABBOTSZLHS']


In [142]:
"""
prod env
"""
# df0 = ld_dt(if_local = True)
# min_kernel = 3
# max_kernel = 11
df_final = pd.DataFrame()
for ou_code in df['ou_code'].unique(): # --
    df = df0[df0['ou_code'] == ou_code]
    ou_code = ou_code #'STARBSYXXS', 'FORDCCSLGS' 
    print(ou_code)
    try:
        del df_train,df_nulle,df_teste
    except:
        pass
    df_train = df[(df['ou_code'] == ou_code) \
        # & (df['operation_day'] >=  20220101)
        & (df['operation_day'] <=  20220310)
        & ((df['inbound_receive_qty'] > 0)
        | (df['outbound_shipped_qty'] > 0))] # train data for fit, either in or out larger than 0 

    df_nulle = df[(df['ou_code'] == ou_code) \
        # & (df['operation_day'] <  20220201)
        & (df['inbound_receive_qty'] == 0) 
        & (df['outbound_shipped_qty'] == 0)] # null data

    df_teste = df[(df['ou_code'] == ou_code) \
        # & (df['operation_day'] <  20220321)
        & ((df['inbound_receive_qty'] > 0) 
        | (df['outbound_shipped_qty'] > 0))]  # none null data, test set, all time span
    try:
        df_teste['kernels'] = train(df_train)
        df_nulle['kernels'] = -1
        df_teste = transform_inout_to_medianlog(df_teste)
        df_nulle = transform_inout_to_medianlog(df_nulle)
        
        df_fin = pd.concat([df_teste, df_nulle], axis = 0).sort_values('operation_day')
        df_fin = df_fin.join(df_fin[[
            'kernels', 'total_working_hour', 'outsource_working_hour'
            ]].groupby(
                'kernels'
                ).transform('quantile', (.75))\
                    .rename({'total_working_hour':'qt_75_wh', 'outsource_working_hour':'qt_75_os'}, axis = 1))
        
        df_fin['qt_75_wh_modified'] = df_fin['qt_75_wh'] * df_fin['std_inout']
        df_final = df_final.append(df_fin)
    except:
        print(ou_code, 'has failed in this conversation')
        
df_final = df_final.fillna(0)

STARBCDXXS
single point in cluster! dropping, center size = 5
single point in cluster! dropping, center size = 6
single point in cluster! dropping, center size = 7
single point in cluster! dropping, center size = 8
single point in cluster! dropping, center size = 9
single point in cluster! dropping, center size = 10
[0.9314232688295228, 0.7691341298408206, 100, 100, 100, 100, 100, 100] 75.21256967483379
4 Center count, cap is 11
COSTASHHTS
single point in cluster! dropping, center size = 9
single point in cluster! dropping, center size = 10
[0.6413934902539441, 0.7769355257929403, 0.7186750323242812, 0.7463043064838962, 0.7367555975766713, 0.7225311125413454, 100, 100] 25.542824383121637
3 Center count, cap is 11
COSTASHH2S
single point in cluster! dropping, center size = 3
single point in cluster! dropping, center size = 4
single point in cluster! dropping, center size = 5
single point in cluster! dropping, center size = 6
single point in cluster! dropping, center size = 7
single poin

In [149]:
# df_final.to_csv('./starb2.csv', index = None)
# df_final[['qt_75_wh_modified', 'kernels']].drop_duplicates()

# test = df_final[(df_final['ou_code']== 'STARBCDXXS') & df_final['in_out_sum'] != 0]
df_final

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,...,location_usage_rate,location_idle_rate,in_out_sum,tt_wh,kernels,in_out_sum_log,std_inout,qt_75_wh,qt_75_os,qt_75_wh_modified
3363,STARBCDXXS,20211101,75224,0,65005,13.0,124,20.00,104.27,0.0000,...,0.247048,0.752952,140229.0,124.0,0,11.989306,1.064538,120.25,29.6650,128.010642
14166,STARBCDXXS,20211102,34189,0,78456,14.0,132,27.73,104.88,0.0000,...,0.240952,0.759048,112645.0,132.0,3,11.615392,0.779877,123.00,35.4700,95.924828
3855,STARBCDXXS,20211103,49174,0,90786,12.0,109,17.12,92.79,0.0000,...,0.236952,0.763048,139960.0,109.0,1,11.899130,0.991842,114.00,25.4900,113.070012
11516,STARBCDXXS,20211104,81363,0,84669,13.0,107,16.63,90.76,0.0000,...,0.239429,0.760571,166032.0,107.0,0,11.989306,1.064538,120.25,29.6650,128.010642
7583,STARBCDXXS,20211105,90494,0,69908,14.0,118,29.53,88.88,0.0000,...,0.232381,0.767619,160402.0,118.0,0,11.989306,1.064538,120.25,29.6650,128.010642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,STARBGZHPS,20220328,259096,0,159956,53.0,365,178.64,117.13,69.8087,...,0.713289,0.286711,419052.0,365.0,0,13.095429,1.108767,505.00,430.5525,559.927383
9414,STARBGZHPS,20220329,119607,0,130370,60.0,421,196.20,115.50,109.6777,...,0.702215,0.297785,249977.0,421.0,4,12.437914,0.765283,350.50,215.5950,268.231789
12728,STARBGZHPS,20220330,141511,0,171913,54.0,377,170.04,88.89,118.2385,...,0.690989,0.309011,313424.0,377.0,4,12.437914,0.765283,350.50,215.5950,268.231789
18461,STARBGZHPS,20220331,165583,0,126642,49.0,371,200.60,119.00,51.5732,...,0.670965,0.329035,292225.0,371.0,4,12.437914,0.765283,350.50,215.5950,268.231789


# preview data

In [145]:


df_cap = df_final.groupby(['ou_code', 'kernels'],as_index = False).agg(
    {
        'in_out_sum' : ['mean', 'median', median_log],
        'qt_75_wh': ['max']
    }
    ).sort_values(
        [
            'ou_code',
            ('in_out_sum', 'median'),
        ]
        )
    
df_cap.columns = ['ou_code', 'kernels','in_out_sum_mean', 'in_out_sum_median', 'in_out_sum_log', 'qt_75_wh']
df_cap['std_inout'] = (StandardScaler().fit_transform(reshape_to_easy_train(df_cap, 'in_out_sum_log'))/10 + 1)**2
df_cap['new'] = df_cap['qt_75_wh'] * df_cap['std_inout']
df_cap

ValueError: math domain error

In [129]:
df_fin[[
    'operation_day',
    'total_working_hour', 
    'inbound_receive_qty', 
    'outbound_shipped_qty', 
    'qt_75_wh', 
    'kernels',
    'in_out_sum',
    'qt_75_wh_modified'
    ]].sort_values('in_out_sum', ascending = False).head(50)

,operation_day,total_working_hour,inbound_receive_qty,outbound_shipped_qty,qt_75_wh,kernels,in_out_sum,qt_75_wh_modified
10873,20220207,218,238865,758758,418.0,3,997623.0,553.777112
9770,20211224,363,354702,584403,418.0,3,939105.0,553.777112
5544,20220103,287,626478,273742,418.0,3,900220.0,553.777112
4641,20211222,361,457020,387967,418.0,3,844987.0,553.777112
1699,20211101,705,592124,244007,418.0,3,836131.0,553.777112
15603,20211213,319,572753,223211,418.0,3,795964.0,553.777112
1012,20220124,418,513589,277324,418.0,3,790913.0,553.777112
7672,20211108,675,537208,252989,418.0,3,790197.0,553.777112
14523,20211227,351,425965,330401,418.0,3,756366.0,553.777112
1309,20211217,314,494599,224000,418.0,3,718599.0,553.777112


In [41]:
import  matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px


plt = df_fin[['inbound_receive_qty', 'outbound_shipped_qty',  'kernels', 'in_out_sum', 'tt_wh',]]
# sns.scatterplot(x = plt.iloc[:,0],
#                 y = plt.iloc[:,1],
#             hue =plt.iloc[:,2],
#             palette = 'Paired'
#             )
# # pd.DataFrame(input)
px.scatter_3d(data_frame= plt,
              x = 'inbound_receive_qty',
              y = 'outbound_shipped_qty',
              z = 'tt_wh',
              color='kernels',)

# pd.Series(model).value_counts().min()

In [ ]:

df_fin['flag_75_os'] =  df_fin['outsource_working_hour'] -1.2*df_fin['qt_75_os']
df_fin['flag_75_os'] = [1 if a > 0 else 0 for a in df_fin['flag_75_os']]
df_fin['flag_75_wh'] =  df_fin['total_working_hour'] -1.2*df_fin['qt_75_wh']
df_fin['flag_75_wh'] = [1 if a > 0 else 0 for a in df_fin['flag_75_wh']]
df_fin['dis_tt_kernel'] = np.abs(df_fin['total_working_hour'    ]  - 1.2* df_fin['qt_75_wh'] )
df_fin['dis_os_kernel'] = np.abs(df_fin['outsource_working_hour']  - 1.2* df_fin['qt_75_os'] )


# statistical inferencing

In [49]:
# import sklearn
# from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
mod = sm.OLS(
        endog=df['total_working_hour'],
        exog=df[['inbound_receive_qty','outbound_shipped_qty']])
fii = mod.fit()
# p_values = fii.summary2().tables[1]['P>|t|']
# p_values
fii.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
inbound_receive_qty,0.0009,0.000,6.351,0.000,0.001,0.001
outbound_shipped_qty,0.0013,0.000,8.849,0.000,0.001,0.002


In [ ]:
# df_fin[['inbound_receive_qty','outbound_shipped_qty', 'kernels']].query("kernels == 2")

In [ ]:
# select a.ou_code,b.ou_name,b.bg_code,c.bg_name_cn bg_name,operation_day,inbound_receive_qty,outbound_shipped_qty,

# round(total_working_hour,2) total_working_hour,

# round(1.2*qt_75_wh,2) working_hour_limit,round(dis_tt_kernel,2) dis_tt_kernel

In [ ]:

# a = cluster.DBSCAN()
# get_transformed_data('train')
# input = get_transformed_data('train')
# scores = []
# for center in range(3,6):
#         #  np.linspace(0.1, input.max()/2, num=5)
#                 # kmeans = cluster.DBSCAN(eps=center, min_samples=7, n_jobs= -1)
#                 # kmeans = cluster.SpectralClustering(n_clusters=center,\
#                 #         affinity='nearest_neighbors',
#                 #         n_neighbors=4,
#                 #         random_state=529, 
#                 #         n_jobs= -1)
#                 kmeans = cluster.KMeans(n_clusters=center, random_state = 529)
#                 model  = kmeans.fit_predict(
#                         # X = reshape_to_easy_train(df_train, col)
#                         X = input
#                         )
#                 score  = davies_bouldin_score(
#                         # reshape_to_easy_train(df_train, col), 
#                         input,
#                         model
#                         )
#                 scores.append(score)
# scores
# # np.argmin(scores) + 4
# # kmeans = cluster.DBSCAN(eps=.15, min_samples=4, n_jobs= -1)
# # kmeans = cluster.SpectralClustering(n_clusters=4, 
# #                                     affinity='nearest_neighbors',
# #                                     n_neighbors=4,
# #                                     random_state=529, 
# #                                     n_jobs= -1)
# kmeans = cluster.KMeans(n_clusters=np.argmin(scores) + 3)
# model  = kmeans.fit_predict(
#                         # X = reshape_to_easy_train(df_train, col)
#                         X = input
#                         )
# # model.tolist()